## Building docker container images

To build the Dockerfile inside SageMaker Studio using sagemaker-studio-image-build cli, you also need the following policy attached to your execution role. See more in [Using the Amazon SageMaker Studio Image Build CLI to build container images from your Studio notebooks](https://aws.amazon.com/blogs/machine-learning/using-the-amazon-sagemaker-studio-image-build-cli-to-build-container-images-from-your-studio-notebooks/).

In [ ]:
!pip install -q sagemaker-studio-image-build

In [ ]:
!sm-docker build -h

In [ ]:
# need testing
# %%sh
!sm-docker build . --repository sagemaker-studio-alphafold:processor --file ./docker/Dockerfile.processor

In [ ]:
!cd docker;sm-docker build . --repository sagemaker-studio-alphafold:estimator --file ./Dockerfile.estimator \
                   --compute-type BUILD_GENERAL1_MEDIUM
# small causes out of memory

## Setup SageMaker Session & User Environment Inputs

In [ ]:
import time
from time import gmtime, strftime
import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.inputs import FileSystemInput
from sagemaker.estimator import Estimator

client = boto3.client("sts")
account=client.get_caller_identity()["Account"]

sess=sagemaker.Session()
region = boto3.session.Session().region_name
role = get_execution_role()
default_bucket=sess.default_bucket()

In [ ]:
#User Input - Static Envrionment Variables

prefix='xxxxxx'

# Specify FSx Lustre file system id.
file_system_id = 'xxxxxx'
# Specify FSx Lustre mount id.
fsx_mount_id = 'xxxxxx' 

# The same VPC/subnet where FSx for Lustre is hosted
vpc_subnet_ids = ['xxxxxx'] 
security_group_ids = ['xxxxxx']

#Below variables do not need to be changed if using default settings
file_system_access_mode = 'ro'
file_system_type = 'FSxLustre'
file_system_directory_path = f'/{fsx_mount_id}/{prefix}/alphafold-genetic-db'

## Prepare protein sequence fasta files

In [ ]:
!mkdir ./sequence_input/
!curl 'https://www.predictioncenter.org/casp14/target.cgi?target=T1030&view=sequence' > ./sequence_input/T1030.fasta 

## Run MSA with genetic db on FSx for Lustre

In [ ]:
!pygmentize source_dir/run_create_alignment.sh

In [ ]:
alphafold_image_uri=f'{account}.dkr.ecr.{region}.amazonaws.com/sagemaker-studio-alphafold:v2.3.0-estimator'

In [ ]:
genetic_db = FileSystemInput(
    file_system_id=file_system_id,
    file_system_type=file_system_type,
    directory_path=file_system_directory_path,
    file_system_access_mode=file_system_access_mode
)

s3_fasta=sess.upload_data(path='sequence_input/T1030.fasta',
                          key_prefix='alphafoldv2/sequence_input')
fasta = sagemaker.inputs.TrainingInput(s3_fasta,
                                       distribution="FullyReplicated", 
                                       s3_data_type="S3Prefix",
                                       input_mode='File'
                                      )

data_channels_msa = {"genetic": genetic_db, 'fasta': fasta}

In [ ]:
db_preset='full_dbs'
parameters={
            'DB_PRESET': db_preset, 
            'FASTA_SUFFIX': 'T1030.fasta',
            'MAX_TEMPLATE_DATE': '2020-05-14',
            'MODEL_PRESET': 'monomer',
            'NUM_MULTIMER_PREDICTIONS_PER_MODEL': '5',
           }

output_path='s3://%s/%s/job-output/'%(default_bucket, prefix)

run_msa = Estimator( 
                      source_dir='./source_dir',
                      entry_point='run_create_alignment.sh',
                      role=role,
                      image_uri=alphafold_image_uri,
                      instance_count=1,
                      instance_type='ml.m5.4xlarge',
                      volume_size=3000,
                      sagemaker_session=sess,
                      subnets=vpc_subnet_ids,
                      security_group_ids=security_group_ids,
                      debugger_hook_config=False,
                      base_job_name='msa-default-run',
                      hyperparameters=parameters,
                      enable_sagemaker_metrics=True,
                      output_path=output_path)

In [ ]:
run_msa.fit(inputs=data_channels_msa,
              wait=False,
              )

## Run Alphafold with MSA Output

In [ ]:
!pygmentize source_dir/run_alphafold.sh

In [ ]:
msa = sagemaker.inputs.TrainingInput(run_msa.model_data,
                                     distribution="FullyReplicated", 
                                     s3_data_type="S3Prefix",
                                     input_mode='File'
                                    )

data_channels_alphafold = {"genetic": genetic_db, 'fasta': fasta, 'msa': msa}
print(data_channels_alphafold)

In [ ]:
instance_type='ml.g5.2xlarge'
instance_count=1
    
db_preset='full_dbs'
parameters={
            'DB_PRESET': db_preset, # <full_dbs|reduced_dbs>
            'FASTA_SUFFIX': 'T1030.fasta',
            'MAX_TEMPLATE_DATE': '2020-05-14',
            'MODEL_PRESET': 'monomer',
            'NUM_MULTIMER_PREDICTIONS_PER_MODEL': '5',
           }
output_path='s3://%s/%s/job-output/'%(default_bucket, prefix)

estimator_alphafold = Estimator(
                      source_dir='source_dir',
                      entry_point='run_alphafold.sh',
                      role=role,
                      image_uri=alphafold_image_uri,
                      instance_count=instance_count,
                      instance_type=instance_type,
                      sagemaker_session=sess,
                      subnets=vpc_subnet_ids,
                      security_group_ids=security_group_ids,
                      debugger_hook_config=False,
                      base_job_name='alphafold-run-fsx',
                      hyperparameters=parameters,
                      keep_alive_period_in_seconds=3600,
                      max_run=172800,
                      enable_sagemaker_metrics=True,
                      code_location=output_path,
                      output_path=output_path)

In [ ]:
estimator_alphafold.fit(inputs=data_channels_alphafold,
              wait=False,
              )

# Run OpenFold with MSA Output

In [ ]:
!git clone -b v1.0.1 --single-branch https://github.com/aqlaboratory/openfold.git

In [ ]:
!cd ~/openfold
!sm-docker build . --repository sagemaker-studio-openfold:base-v1.0.1 --file ./Dockerfile --compute-type BUILD_GENERAL1_MEDIUM

In [ ]:
!cd ~/protein-folding-on-sagemaker/docker
!sm-docker build . --repository sagemaker-studio-openfold:v1.0.1 --file ./Dockerfile.openfold 

In [ ]:
openfold_image_uri=f'{account}.dkr.ecr.{region}.amazonaws.com/sagemaker-studio-openfold:v1.0.1'

In [ ]:
!pygmentize source_dir/run_openfold.sh

In [ ]:
# TODO: show how to download the openfold_params/ from RODA
s3_param=sess.upload_data(path='./source_dir/finetuning_ptm_2.pt',
                          key_prefix=f'{prefix}/openfold_params')
param = sagemaker.inputs.TrainingInput(s3_param,
                                       distribution="FullyReplicated", 
                                       s3_data_type="S3Prefix",
                                       input_mode='File'
                                      )

data_channels_openfold = {'genetic': genetic_db, 
                          'fasta': fasta, 
                          'param': param,
                          'msa' : msa
                         }

In [ ]:
instance_type='ml.g5.2xlarge'
instance_count=1

db_preset='full_dbs'
parameters={
            'DB_PRESET': db_preset, # <full_dbs|reduced_dbs>
           }
output_path='s3://%s/%s/job-output'%(default_bucket, prefix)

estimator_openfold = Estimator(source_dir='./source_dir',
                              entry_point='run_openfold.sh',
                              role=role,
                              image_uri=openfold_image_uri,
                              instance_count=instance_count,
                              instance_type=instance_type,
                              sagemaker_session=sess,
                              subnets=vpc_subnet_ids,
                              security_group_ids=security_group_ids,
                              debugger_hook_config=False,
                              base_job_name='openfold-run-fsx',
                              hyperparameters=parameters,
                              keep_alive_period_in_seconds=3600,
                              enable_sagemaker_metrics=True,
                              output_path=output_path,
                              code_location=output_path)

In [ ]:
estimator_openfold.fit(inputs=data_channels_openfold,
                       wait=False,
                      )

## Accessing predicted structure and visualize
Ref: https://github.com/aws-solutions-library-samples/aws-batch-arch-for-protein-folding/blob/0a0a62faa0d407f4856e0865df4c8e8ec6d26290/src/batchfold/utils/utils.py#LL102C5-L102C5

In [ ]:
!pip install -q py3dmol

In [ ]:
!pip install dm-tree

In [ ]:
!pip install biopython

In [ ]:
import sys, pymol
from pymol import cmd, stored
import py3Dmol
import matplotlib.pyplot as plt

from Bio.PDB import PDBParser, PDBIO
import io
from source_dir import protein
from source_dir import residue_constants

In [ ]:
def overwrite_b_factors(pdb_str: str, bfactors: np.ndarray) -> str:
    """Overwrites the B-factors in pdb_str with contents of bfactors array.

    Args:
      pdb_str: An input PDB string.
      bfactors: A numpy array with shape [1, n_residues, 37]. We assume that the
        B-factors are per residue; i.e. that the nonzero entries are identical in
        [0, i, :].

    Returns:
      A new PDB string with the B-factors replaced.
    """
    if bfactors.shape[-1] != residue_constants.atom_type_num:
        raise ValueError(
            f"Invalid final dimension size for bfactors: {bfactors.shape[-1]}."
        )

    parser = PDBParser(QUIET=True)
    handle = io.StringIO(pdb_str)
    structure = parser.get_structure("", handle)

    curr_resid = ("", "", "")
    idx = -1
    for atom in structure.get_atoms():
        atom_resid = atom.parent.get_id()
        if atom_resid != curr_resid:
            idx += 1
            if idx >= bfactors.shape[0]:
                raise ValueError(
                    "Index into bfactors exceeds number of residues. "
                    "B-factors shape: {shape}, idx: {idx}."
                )
        curr_resid = atom_resid
        atom.bfactor = bfactors[idx, residue_constants.atom_order["CA"]]

    new_pdb = io.StringIO()
    pdb_io = PDBIO()
    pdb_io.set_structure(structure)
    pdb_io.save(new_pdb)
    return new_pdb.getvalue()

def plot_banded_pdb(pdb_file, show_sidechains = False, width = 800, height = 600):
    with open(pdb_file) as f:
            best_pdb = f.read()
    target_protein = protein.from_pdb_string(best_pdb)
    plddt_list = target_protein.b_factors[:,0]
    atom_mask = target_protein.atom_mask
    banded_b_factors = []
    for plddt in plddt_list:
        for idx, (min_val, max_val, _) in enumerate(residue_constants.PLDDT_BANDS):
            if plddt >= min_val and plddt <= max_val:
                banded_b_factors.append(idx)
                break

    banded_b_factors = (
            np.array(banded_b_factors)[:, None] * atom_mask
    )

    to_visualize_pdb = overwrite_b_factors(best_pdb, banded_b_factors)
    # Color the structure by per-residue pLDDT
    color_map = {i: bands[2] for i, bands in enumerate(residue_constants.PLDDT_BANDS)}
    view = py3Dmol.view(width, height)
    view.addModelsAsFrames(to_visualize_pdb)
    style = {"cartoon": {"colorscheme": {"prop": "b", "map": color_map}}}
    if show_sidechains:
        style["stick"] = {}
    view.setStyle({"model": -1}, style)
    view.zoomTo()
    view.show()
    return None

def plot_plddt_legend():
    """Plots the legend for pLDDT."""

    thresh = [
        "Very low (pLDDT < 50)",
        "Low (70 > pLDDT > 50)",
        "Confident (90 > pLDDT > 70)",
        "Very high (pLDDT > 90)",
    ]

    colors = [x[2] for x in residue_constants.PLDDT_BANDS]

    plt.figure(figsize=(2, 2))
    for c in colors:
        plt.bar(0, 0, color=c)
    plt.legend(thresh, frameon=False, loc="center", fontsize=20)
    plt.xticks([])
    plt.yticks([])
    ax = plt.gca()
    ax.spines["right"].set_visible(False)
    ax.spines["top"].set_visible(False)
    ax.spines["left"].set_visible(False)
    ax.spines["bottom"].set_visible(False)
    plt.title("Model Confidence", fontsize=20, pad=20)
    return plt

### Alphafold output

In [ ]:
estimator_alphafold.model_data

In [ ]:
!aws s3 cp {estimator_alphafold.model_data} .

In [ ]:
!tar zxfv model.tar.gz

In [ ]:
plt=plot_plddt_legend()
plt.show()
plot_banded_pdb('T1030/ranked_0.pdb')

### Openfold output

In [ ]:
estimator_openfold.model_data

In [ ]:
!aws s3 cp {estimator_openfold.model_data} openfold_output/model.tar.gz
!tar zxfv openfold_output/model.tar.gz -C openfold_output/

In [ ]:
plt=plot_plddt_legend()
plt.show()
plot_banded_pdb('openfold_output/predictions/T1030_model_1_ptm_relaxed.pdb')

### Compare against the structure in RCSB 
Ref: https://notebook.community/aloctavodia/SBioA/English/04_Comparing_structures

In [ ]:
# get T1030 structure from RCSB
!curl https://files.rcsb.org/view/6POO.pdb >> sequence_input/6POO.pdb

In [ ]:
import pymol
from pymol import cmd, stored
import numpy as np

def rmsd_cur(mol0, mol1, sel='*'):
    """
    Computes the root mean square deviation from the current
    coordinates of two pairs of equivalent atoms. Does not
    perform a superposition.
    
    Parameters
    ----------
    mol0 : PyMOL object
    mol1 : PyMOL object
    sel  : PyMOL selection, atoms used to compute rmsd.
           e.g. use ca+c+n for the backbone
    """
    model0 = cmd.get_model('%s and name %s' % (mol0, sel))
    model1 = cmd.get_model('%s and name %s'  % (mol1, sel))
    xyz0 = np.array(model0.get_coord_list())
    xyz1 = np.array(model1.get_coord_list())
    
    rmsd = (np.sum((xyz0 - xyz1 )**2)/len(xyz0))**0.5
    return rmsd


def rmsd_fit(mol0, mol1, sel='*', fit=True):
    """
    Computes the root mean square deviation from two pairs of
    equivalent atoms after superposition.
    
    Parameters
    ----------
    mol0 : PyMOL object
    mol1 : PyMOL object
    sel  : PyMOL selection. atoms used to compute rmsd.
           e.g. use ca+c+n for the backbone
    fit  : bool. If false computes the rmsd after superposition, but without
           updating the coordinates
           
    """
    xyz0 = np.array(cmd.get_model('%s and name %s' % (mol0, sel)).get_coord_list())
    xyz1 = np.array(cmd.get_model('%s and name %s'  % (mol1, sel)).get_coord_list())
    
    xyz0_all = np.array(cmd.get_model('%s' % mol0).get_coord_list())
    xyz1_all = np.array(cmd.get_model('%s'  % mol1).get_coord_list())
    
    # Translation
    X = xyz0 - xyz0.mean(axis=0)
    Y = xyz1 - xyz1.mean(axis=0)
    # Covariation matrix
    Cov_matrix = np.dot(Y.T, X)
    # Optimal rotation matrix
    U, S, Wt = np.linalg.svd(Cov_matrix)
    # Create Rotation matrix R
    R = np.dot(U, Wt)
    # Ensure a right-handed coordinate system
    if np.linalg.det(R) < 0.:
        S[-1] = -S[-1]
        Wt[-1] *= -1
        R = np.dot(U, Wt) 
    if fit:
        # center the first molecule
        stored.sel0 = list(xyz0_all - xyz0.mean(axis=0))
        # rotate and translate the second molecule
        stored.sel1 = list(np.dot((xyz1_all - xyz1.mean(0)), R))
        #update the changes to the coordinates 
        cmd.alter_state(1, mol0,"(x,y,z)=stored.sel0.pop(0)")
        cmd.alter_state(1, mol1,"(x,y,z)=stored.sel1.pop(0)")

    # We compute the RMSD after superposition by using the matrix S. The advantage is 
    # we do not need to actually do the superposition before computing the RMSD.
    #rmsd = (np.exp(np.log(np.sum(X ** 2) + np.sum(Y ** 2)) - 2.0 * np.log(np.sum(S)))/len(X))**0.5
    rmsd = ((np.sum(X ** 2) + np.sum(Y ** 2) - 2.0 * np.sum(S))/len(X))**0.5
    # scales and translates the window to show a selection
    cmd.zoom()
    return rmsd


def tm_score(mol0, mol1, sel='*'): #Check if TM-align use all atoms!
    """
    Compute TM-score between two set of coordinates
    
    Parameters
    ----------
    mol0 : PyMOL object
    mol1 : PyMOL object
    sel  : PyMOL selection, atoms used to compute rmsd.
           e.g. use ca+c+n for the backbone
    """
    xyz0 = np.array(cmd.get_model('%s and name %s' % (mol0, sel)).get_coord_list())
    xyz1 = np.array(cmd.get_model('%s and name %s'  % (mol1, sel)).get_coord_list())
    
    L = len(xyz0)
    # d0 is less than 0.5 for L < 22 
    # and nan for L < 15 (root of a negative number)
    d0 = 1.24 * np.power(L - 15, 1/3) - 1.8
    d0 = max(0.5, d0) 

    # compute the distance for each pair of atoms
    di = np.sum((xyz0 - xyz1) ** 2, 1) # sum along first axis
    return np.sum(1 / (1 + (di / d0) ** 2)) / L

In [ ]:
cmd.load('T1030/ranked_0.pdb')
cmd.load('sequence_input/6POO.pdb')
cmd.remove('not polymer or hydro')
object1 = cmd.get_names()[0]
object2 = cmd.get_names()[1]
print(object1)
print(object2)

# compute rmsd_cur
rmsd = rmsd_cur(object1, object2, sel='ca') #'ca+c+n'
print('RMSD: %.2f' % rmsd)

# compute rmsd_fit
rmsd_sp = rmsd_fit(object1, object2, sel='ca', fit=True)
print('RMSD superpositioned: %.2f' % rmsd_sp)

# compute TM score
tmscore = tm_score(object1, object2, sel='ca')
print('TM Score: %.4f' % tmscore)